#Package Section

In [2]:
import sys
import numpy as np
import copy
from numpy import linalg as LA
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics

#Classes and functions

In [3]:
# Supress/hide the warning
# invalide devide resutls will be nan
np.seterr(invalid='ignore')
############------------Auto_select_method_start-----------------###############
def Run(case, opt, **kwargs):
  """
    input X can be a list of one of these format below:
    1. python list of n*n adjacency matrices.
    2. python list of s*2 edge lists. 
    3. python list of s*3 edge lists. 
    input Y can be these choices below:
    1. no Y input. The default will be [2,3,4,5] -- K range for clusters.
    2. n*1 class -- label vector. Positive labels are knwon labels and -1 indicate unknown labels.
    3. A range of potential number of clusters -- K (K clusters in total), i.e., [3, 4, 5].
    
    if input X is n*n adjacency =>  s*3 edg list
    if input X is s*2 => s*3 edg list
    
    Vertex size should be >10.

    Clustering / Classification
    The program automaticlly decide to run clustering or classification.
    1. If Y is a given cluster range, do clustering (case 1,3 for Y).
    2. If Y is a label vector (case 2 for Y), do classification.
    For classification: semi-supervised learning, supervised learning methods. 
                        see the "Learner" defined below. 

    
    "Learner": 
      Learner=0 means embedding via known label, do not learn the unknown labels. 
        a. If all nodes in the training set has known label, 
           it runs supervised learning, will be evaluated on a test set provided
        b. If only some nodes in the training set has known label, 
           the test set is the unknwon labeled set, which is compared with 
           the original labels of the unknown set
      Learner=1 means embedding via partial label, then learn unknown label via LDA.
        this runs semi-supervised learning with NN, 
        the test will be on the result labels with the original labels

      Learner=2 means embedding via partial label, then learn unknown label via two-layer NN.
        this runs semi-supervised learning with NN, 
        the test will be on the result labels with the original labels
    

  """
  defaultKwargs = {'Y':[2,3,4,5], 'DiagA': True,'Correlation': True,'Laplacian': False,
                  'Learner': 1, 'LearnerIter': False, 'MaxIter': 50, 'MaxIterK': 5,
                  'Replicates': 3, 'Attributes': False, 'neuron': 20, 'activation': 'relu'} 
  kwargs = { **defaultKwargs, **kwargs }

  eval = Evaluation()
  Dataset = DataPreprocess(case)
  
  Y = case.Y
  n = case.n

  # auto check block
  # if the option is not clustering, but the Y does not contain labels (known/unknwon) for n nodes. 
  if (opt != "c") and (len(Y) != n):
    opt = "c" # do clustering
    print("The given Y do not have the same size as the node.Y is assumed as cluster number range.",
    "Clustering will be performed.",
    "If you want to do classification, stop the current run, reimport the Y with the right format then run again.",
    sep = "\n")
  # if the option is semi-supervised learning, but there is no unknwon label(-1) in Y
  if (opt == "se") and (-1 not in Y):
    opt = "su" # do supervised-learning
  # if the option is supervised-learning but there is unknwon label in the training set Y.
  if (opt == "su") and (-1 in Y):
    opt = "se" # do semi-supervised-learning  

 
  # clustering
  if opt == 'c':  
    cluster = Clustering(Dataset)
    Z, Y, W, meanSS = cluster.cluster_main()
    ari = eval.clustering_test(Y, case.Y_ori)
    print("ARI: ", ari)

  # supervised learning
  if opt == "su":
    Dataset = Dataset.supervise_preprocess(case)
    kwargs_for_gnn =  {'Learner': 0, 'LearnerIter': -1}
    gnn = GNN(Dataset, **kwargs_for_gnn)
    Results = gnn.GNN_complete()
    acc = eval.GNN_supervise_test(gnn, Dataset.z_test, Dataset.Y_test)
    print("acc: ", acc)
  
  # semisupervised learning
  if opt == "se":
    Dataset = Dataset.semi_supervise_preprocess()
    kwargs_for_gnn = {k: kwargs[k] for k in ['Learner', 'LearnerIter']}
    gnn = GNN(Dataset, **kwargs_for_gnn)
    Results = gnn.GNN_complete()
    if kwargs['Learner'] == 2:
      acc = eval.GNN_semi_supervised_learn_test(Results.Y, case.Y_ori)
    if kwargs['Learner'] == 0:
      acc = eval.GNN_semi_supervised_not_learn_test(gnn, Dataset, case)
    print("acc: ", acc)

  

############------------graph_encoder_embed_start----------------###############
def graph_encoder_embed(X,Y,n,**kwargs):
  """
    input X is s*3 edg list: nodei, nodej, connection weight(i,j)
    graph embedding function
  """
  defaultKwargs = {'Correlation': True}
  kwargs = { **defaultKwargs, **kwargs}

  #If Y has more than one dimention , Y is the range of cluster size for a vertex. e.g. [2,10], [2,5,6]
  # check if Y is the possibility version. e.g.Y: n*k each row list the possibility for each class[0.9, 0.1, 0, ......]
  possibility_detected = False
  if Y.shape[1] > 1:
    k = Y.shape[1]
    possibility_detected = True
  else:
    # assign k to the max along the first column
    # Note for python, label Y starts from 0. Python index starts from 0. thus size k should be max + 1
    k = Y[:,0].max() + 1

  #nk: 1*n array, contains the number of observations in each class
  #W: encoder marix. W[i,k] = {1/nk if Yi==k, otherwise 0}
  nk = np.zeros((1,k))
  W = np.zeros((n,k))

  if possibility_detected:
    # sum Y (Y is a vector, single row in numpy), then do element divid nk.
    nk=np.sum(Y, axis=0)
    W=Y/nk
  else:
    for i in range(k):
      nk[0,i] = np.count_nonzero(Y[:,0]==i)

    for i in range(Y.shape[0]):
      k_i = Y[i,0]
      if k_i >=0:
        W[i,k_i] = 1/nk[0,k_i]

  # Edge List Version in O(s)
  Z = np.zeros((n,k))
  i = 0
  for row in X:
    [v_i, v_j, edg_i_j] = row
    v_i = int(v_i)
    v_j = int(v_j)
    if possibility_detected:
      for label_j in range(k):
        Z[v_i, label_j] = Z[v_i, label_j] + W[v_j, label_j]*edg_i_j
        if v_i != v_j:
          Z[v_j, label_j] = Z[v_j, label_j] + W[v_i, label_j]*edg_i_j
    else:
      label_i = Y[v_i][0] 
      label_j = Y[v_j][0]

      if label_j >= 0:
        Z[v_i, label_j] = Z[v_i, label_j] + W[v_j, label_j]*edg_i_j
      if (label_i >= 0) and (v_i != v_j):
        Z[v_j, label_i] = Z[v_j, label_i] + W[v_i, label_i]*edg_i_j
  
  # Calculate each row's 2-norm (Euclidean distance). 
  # e.g.row_x: [ele_i,ele_j,ele_k]. norm2 = sqr(sum(2^2+1^2+4^2))
  # then divide each element by their row norm
  # e.g. [ele_i/norm2,ele_j/norm2,ele_k/norm2]
  if kwargs['Correlation']:
    row_norm = LA.norm(Z, axis = 1)
    reshape_row_norm = np.reshape(row_norm, (n,1))
    Z = np.nan_to_num(Z/reshape_row_norm)

  return Z, W

def multi_graph_encoder_embed(DataSets, Y):
  """
    input X contains a list of s3 edge list
    get Z and W by using graph emcode embedding
    Z is the concatenated embedding matrix from multiple graphs
    if there are attirbutes provided, add attributes to Z
    W is a list of weight matrix Wi
  """

  X = DataSets.X
  n = DataSets.n
  U = DataSets.U
  Graph_count = DataSets.Graph_count
  attributes = DataSets.attributes
  kwargs = DataSets.kwargs

  W = []

  for i in range(Graph_count):
    if i == 0:
      [Z, Wi] = graph_encoder_embed(X[i],Y,n,**kwargs)
    else:
      [Z_new, Wi] = graph_encoder_embed(X[i],Y,n,**kwargs)
      Z = np.concatenate((Z, Z_new), axis=1)
    W.append(Wi)

  # if there is attributes matrix U provided, add U
  if attributes:
    # add U to Z side by side
    Z = np.concatenate((Z, U), axis=1)

  return Z, W

############------------graph_encoder_embed_end------------------###############

############------------DataPreprocess_start---------------------###############
class DataPreprocess:
  def __init__(self, Dataset_input, **kwargs):
    self.kwargs = self.kwargs_construct(**kwargs)
    # Note, since every element in multi-graph list X has the same size and 
    # node index, there will be only one column in Y for the node labels
    self.Y = Dataset_input.Y  
    self.n = Dataset_input.n
    (self.X, self.Graph_count) = self.input_prep(Dataset_input.X)    
    (self.attributes, self.U) = self.check_attributes()


  def kwargs_construct(self, **kwargs):
    defaultKwargs = {'DiagA': True,'Laplacian': False,  #input_prep
                     'Correlation': True,      # graph_encoder_embed
                     'Attributes': False,      # GNN_preprocess
                     }
    kwargs = { **defaultKwargs, **kwargs}  # update the args using input_args
    return kwargs      


  def check_attributes(self):
    """
      return attributes detected flag and attributes U
    """
    kwargs = self.kwargs
    
    Attributes_detected = False
    U = None

    if kwargs["Attributes"]:
      U = kwargs["Attributes"]
      if U.shape[0] == n:
        Attributes_detected = True
      else:
        print("Attributes need to have the same size as the nodes.\
        If n nodes, need n rows")    
    return Attributes_detected, U

  def supervise_preprocess(self, Dataset_input):
    """
      adding test sets for supervised learning    
      this function assumes only one test set
      if there is a list of test set, needs to modify this function
    """
    Y_test = Dataset_input.Y_test  
    X_test = Dataset_input.X_test  

    DataSets = self.semi_supervise_preprocess()
    W = DataSets.W[-1]  # take the last W

    n_test = Dataset_input.n_test
    n_train = Dataset_input.n
    Graph_count_test = 1   

    if X_test.shape[1] <= 3:
      X_test = self.test_edg_list_to_adj(n_test, n_train, X_test)

    z_test = np.matmul(X_test,W)

    DataSets.X_test = X_test
    DataSets.Y_test = Y_test 
    DataSets.z_test = z_test
    DataSets.n_test = n_test 
    DataSets.Graph_count_test = Graph_count_test

    return DataSets

  def test_edg_list_to_adj(self, n_test, n, edg_list):
    adj = np.zeros((n_test,n))

    for row in edg_list:
      [node_i, node_j, edge_i_j] = row
      adj[node_i, node_j] = edge_i_j
    
    return adj


  def input_prep(self, X):
    """
      X may be a single numpy object or a list of numpy objects
      The multi-graph input X is assumed has the same node numbers 
      for each element in X, and the node are indexed the same way 
      amonge the elements. e.g. node_0 in X[1] is the same node_0 in X[2]. 
      return X as a list of s*3 edge lists
      return n, which is the total number of nodes
    """

    # need total labeled number n 
    # if try to get from the edg list, it may miss the node that has no connection with others but has label 
    n = self.n

    # if X is a single numpy object, put this numpy object in a list
    if type(X) == np.ndarray:
      X = [X]
    
    ## Now X is a list of numpy objects
    # each element can be a numpy object for adjacency matrix or edge list

    Graph_count = len(X)    

    for i in range(Graph_count):
      X_tmp = X[i]
      X_tmp = self.to_s3_list(X_tmp) 

      # # Note: now X_tmp is s*3 edg list
      # if i == 0:
      #   # get the number of nodes that has connection with others
      #   # every graph needs to have the same size 
      #   # thus only need to get the node number once
      #   n = int(np.amax(X_tmp[:, :2]) + 1)  
      X_tmp = self.single_X_prep(X_tmp, n)
      X[i] = X_tmp
    
    return X, Graph_count


  def to_s3_list(self,X):
    """
      the input X is a signle graph, can be adjacency matrix or edgelist
      this function will return a s3 edge list
    """
    (s,t) = X.shape

    if s == t:
      # convert adjacency matrix to edgelist
      X = self.adj_to_edg(X);
    else:
      # for either s*2 or s*3 case, calculate n -- vertex number
      if t == 2: 
        # enlarge the edgelist to s*3 by adding 1 to the thrid position as adj(i,j)
        X = np.insert(X, 1, np.ones(s,1))
    return X


  def single_X_prep(self, X, n):
    """
      input X is a single S3 edge list
      this adds Diagnal augement and Laplacian normalization to the edge list
    """
    kwargs = self.kwargs

    # Diagnal augment
    if kwargs['DiagA']:
      # add self-loop to edg list -- add 1 connection for each (i,i)
      self_loops = np.column_stack((np.arange(n), np.arange(n), np.ones(n)))
      # faster than vstack --  adding the second to the bottom
      X = np.concatenate((X,self_loops), axis = 0)
    
    # Laplacian 
    s = X.shape[0] # get the row number of the edg list
    if kwargs["Laplacian"]:
      D = np.zeros((n,1))
      for row in X:
        [v_i, v_j, edg_i_j] = row
        v_i = int(v_i)
        v_j = int(v_j)
        D[v_i] = D[v_i] + edg_i_j
        if v_i != v_j:
          D[v_j] = D[v_j] + edg_i_j

      D = np.power(D, -0.5)

      for i in range(s):
        X[i,2] = X[i,2] * D[int(X[i,0])] * D[int(X[i,1])]
    
    return X

  def adj_to_edg(self,A):
    """
      input is the adjacency matrix: A
      other variables in this function:
      s: number of edges
      return edg_list -- matrix format with shape(edg_sum,3):
      example row in edg_list(matrix): [vertex1, vertex2, connection weight from Adj matrix]
    """
    # check the len of the second dimenson of A
    if A.shape[1] <= 3:
      edg = A
    else:
      n = A.shape[0]
      # construct the initial edgg_list matrix with the size of (edg_sum, 3)
      edg_list = []
      for i in range(n):
        for j in range(n):
          if A[i,j] > 0:
            edg_list.append([i, j, A[i,j]])
      edg = np.array(edg_list)
    return edg

  def semi_supervise_preprocess(self):
    """
      get Z, W using multi_graph_encoder_embed()
      get training sets and testing sets for Z and Y by using split_data()
      
    """
    DataSets =  copy.deepcopy(self)
    Y = DataSets.Y

    (Z, W) = multi_graph_encoder_embed(DataSets, Y)
    
    DataSets.Z = Z
    DataSets.W = W
    DataSets.k = DataSets.get_k() 
    DataSets = DataSets.split_data()
  
    return DataSets


  def get_k(self):
    Y = self.Y
    n = self.n
    # get class number k or the largest cluster size 
    # max of all flattened element + 1  
    if len(Y) == n:
      k = np.amax(Y) + 1   
    return k


  def split_data(self):
    split_Sets =  copy.deepcopy(self)
   
    Y = split_Sets.Y
    Z = split_Sets.Z
    
    ind_train = np.argwhere (Y >= 0)[:,0]
    ind_unlabel = np.argwhere (Y < 0)[:,0]   

    Y_train = Y[ind_train, 0]
    z_train = Z[ind_train]

    Y_unlabel = None
    z_unlabel = None

    if len(ind_unlabel) > 0:
      Y_unlabel = Y[ind_unlabel, 0]
      z_unlabel = Z[ind_unlabel] 


    split_Sets.ind_unlabel = ind_unlabel
    split_Sets.ind_train = ind_train    
    split_Sets.Y_train = Y_train
    split_Sets.Y_unlabel = Y_unlabel
    split_Sets.z_train = z_train
    split_Sets.z_unlabel = z_unlabel

    return split_Sets


  def DataSets_reset(self, option):
    """
      based on the information of the given new Y:
      1. reassign Z and W to the given DataSets, 
      2. update z_train, z_unlabel 
    """
    NewSets =  copy.deepcopy(self)
    kwargs = NewSets.kwargs
    ind_unlabel = NewSets.ind_unlabel
    ind_train = NewSets.ind_train    

    # different versions
    if option == "y_temp":
      [Z,W] = multi_graph_encoder_embed(NewSets, NewSets.y_temp)
    if option == "y_temp_one_hot":
      [Z,W] = multi_graph_encoder_embed(NewSets, NewSets.y_temp_one_hot)  

    if NewSets.attributes:
      # add U to Z side by side
      Z = np.concatenate((Z, NewSets.U), axis=1)  
    
    NewSets.Z = Z
    NewSets.z_train = Z[ind_train]
    NewSets.z_unlabel = Z[ind_unlabel]
    NewSets.y_temp_train_one_hot= NewSets.y_temp_one_hot[ind_train]

    return NewSets


############------------DataPreprocess_end-----------------------###############

############-----------------GNN_start---------------------------###############
class Hyperperameters:
  """
    define perameters for GNN.
    default values are for GNN learning -- "Leaner" ==2:
      embedding via partial label, then learn unknown label via two-layer NN

  """
  def __init__(self):
    # there is no scaled conjugate gradiant in keras optimiser, use defualt instead
    # use whatever default
    self.learning_rate = 0.01  # Initial learning rate.
    self.epochs = 100 #Number of epochs to train.
    self.hidden = 20 #Number of units in hidden layer 
    self.val_split = 0.1 #Split 10% of training data for validation
    self.loss = 'categorical_crossentropy' # loss function


class GNN:
  def __init__(self, DataSets, **kwargs):
    GNN.kwargs = self.kwargs_construct(**kwargs)
    GNN.DataSets = DataSets
    GNN.hyperM = Hyperperameters()
    GNN.model = self.GNN_model()  #model summary: GNN.model.summary()
    GNN.meanSS = 0  # initialize the self-defined critirion meanSS
    
  def kwargs_construct(self, **kwargs):
    defaultKwargs = {'Learner': 1,                    # GNN_Leaner
                     'LearnerIter': 0,                # GNN_complete, GNN_Iter
                     "Replicates": 3                  # GNN_Iter                           
                     }
    kwargs = { **defaultKwargs, **kwargs}  # update the args using input_args
    return kwargs      
 
  
  def GNN_model(self):
    """
      build GNN model
    """
    hyperM = self.hyperM
    DataSets = self.DataSets

    z_train = DataSets.z_train
    k = DataSets.k

    feature_num = z_train.shape[1]
    
    model = keras.Sequential([
    keras.layers.Flatten(input_shape = (feature_num,)),  # input layer 
    keras.layers.Dense(hyperM.hidden, activation='relu'),  # hidden layer -- no tansig activation function in Keras, use relu instead
    keras.layers.Dense(k, activation='softmax') # output layer, matlab used softmax for patternnet default ??? max(opts.neuron,K)? opts 
    ])

    optimizer = keras.optimizers.Adam(learning_rate = hyperM.learning_rate)

    model.compile(optimizer='adam',
                  loss=hyperM.loss,
                  metrics=['accuracy'])

    return model
  

  def GNN_run(self, k, z_train, y_train_one_hot, z_unlabel):
    """
      Train and test directly.
      Do not learn from the unknown labels.
    """
    gnn = copy.deepcopy(self)
    hyperM = gnn.hyperM
    model = gnn.model    

    history = model.fit(z_train, y_train_one_hot, 
          epochs=hyperM.epochs, 
          validation_split=hyperM.val_split,  
          verbose=0)
    
    train_acc = history.history['accuracy'][-1]
    
    predict_probs = None
    pred_class = None
    pred_class_prob = None
    if type(z_unlabel) == np.ndarray:
      # predict_probas include probabilities for all classes for each node
      predict_probs = model.predict(z_unlabel)
      # assign the classes with the highest probability
      pred_class = np.argmax(predict_probs, axis=1)
      # the corresponding probabilities of the predicted classes
      pred_class_prob = predict_probs[range(len(predict_probs)),pred_class]

    gnn.model = model
    gnn.train_acc = train_acc
    gnn.pred_probs = predict_probs  
    gnn.pred_class = pred_class
    gnn.pred_class_prob = pred_class_prob


    return gnn

  def GNN_Direct(self, DataSets, y_train_one_hot):
    """
      This function can run:
      1. by itself, when interation is set to False (<1)
      2. inside GNN_Iter, when interation is set to True (>=1)

      Learner == 0: GNN, but not learn from the known label
      Learner == 2: GNN, and learn unknown labels 
    """
    Learner = self.kwargs["Learner"]  

    k = DataSets.k 
    z_train = DataSets.z_train 
    Y = DataSets.Y
    z_unlabel = DataSets.z_unlabel
    ind_unlabel = DataSets.ind_unlabel

    gnn = self.GNN_run(k, z_train, y_train_one_hot, z_unlabel)

    if Learner == 0:
      # do not learn unknown label.
      pass
    
    if Learner == 2:
      # learn unknown label based on the known label
      # replace the unknown label in Y with predicted labels
      pred_class = gnn.pred_class
      Y[ind_unlabel, 0] = pred_class

    gnn.Y = Y

    return gnn


  def GNN_Iter(self, DataSets):
    """
      Run this function when interation is set, which is >=1.
      
      1. randomly assign labels to the unknown labels, get Y_temp
      2. get Y_one_hot for the Y_temp 
      3. get Z from graph_encod function with X and Y_temp 
      within each loop: 
        use meanSS as its criterion to decide if the update is needed        
	      update Y_one_hot for the unknown labels with predict probabilities of each classes
	      update Y with the highest possible predicted labels
	      update z_train and z_unlabel from graph encoder embedding using the updated Y
	      train the model with the updated z_train and Y_one_hot     
    """

    kwargs = self.kwargs
    meanSS = self.meanSS

    k = DataSets.k 
    z_train = DataSets.z_train 
    Y = DataSets.Y
    z_unlabel = DataSets.z_unlabel
    ind_unlabel = DataSets.ind_unlabel

    y_temp = np.copy(Y)

    for i in range(kwargs["Replicates"]):
      # assign random integers in [1,K] to unassigned labels
      r = [i for i in range(k)]
      
      ran_int = np.random.choice(r, size=(len(ind_unlabel),1))

      y_temp[ind_unlabel] = ran_int
      # Convert targets into one-hot encoded format      
      y_temp_one_hot = to_categorical(y_temp) 
      
      DataSets.y_temp = y_temp
      DataSets.y_temp_one_hot = y_temp_one_hot 
 
      for j in range(kwargs["LearnerIter"]):
        if j ==0:
          # first iteration need to split the y_temp for training etc.
          # use reset to add z_train, z_unlabel, and y_temp_train to the dataset
          DataSets = DataSets.DataSets_reset("y_temp")         
        if j > 0:
          # update z_train, z_unlabel, and y_temp_train_one_hot to the dataset
          DataSets = DataSets.DataSets_reset("y_temp_one_hot")
        # all the gnn train on y_temp_train_one_hot
        gnn = self.GNN_Direct(DataSets, DataSets.y_temp_train_one_hot)
        predict_probs = gnn.pred_probs
        pred_class = gnn.pred_class
        pred_class_prob = gnn.pred_class_prob

        # z_unknown is initialized with none, so the pred_class may be none
        # This will not happen for the semi version,
        # since the unknown size should not be none for the semi version
        if type(pred_class) == np.ndarray:
          # if there are unkown labels and predicted labels are available
          # check if predicted_class are the same as the random integers
          # if so, stop the iteration in "LearnerIter" loop
          # shape (n,) is required for adjusted_rand_score()
          if adjusted_rand_score(ran_int.reshape((-1,)), pred_class) == 1:
            break
          # assign the probabilites for each class to the temp y_one_hot
          DataSets.y_temp_one_hot[ind_unlabel] = predict_probs
          # assgin the predicted classes to the temp Y unknown labels 
          DataSets.y_temp[ind_unlabel, 0] = pred_class 
          # # assign the highest possibility of the class to Y_temp
          # Y_temp[ind_unlabel, 0] = pred_class_prob
      minP = np.mean(pred_class_prob) - 3*np.std(pred_class_prob)
      if minP > meanSS:
        meanSS = minP
        Y = DataSets.y_temp   

      gnn.Y = Y
      gnn.meanSS = meanSS
      return gnn  
  
        
  def GNN_complete(self):
    """
      if LearnerIter set to False(<1):
        run GNN_Direct() with no iteration
      if LearnerIter set to True(>=1):
        run GNN_Iter(), which starts with radomly assigned k to unknown labels
      
    """
    kwargs = self.kwargs

    DataSets = self.DataSets
    y_train = DataSets.Y_train

    if kwargs["LearnerIter"] < 1:
      # Convert targets into one-hot encoded format
      y_train_one_hot = to_categorical(y_train)
      gnn = self.GNN_Direct(DataSets, y_train_one_hot)
    else:
      gnn = self.GNN_Iter(DataSets)
    
    return gnn
############-----------------GNN_end-----------------------------###############

############-----------------LDA_start---------------------------###############
class LDA:
  def __init__(self, DataSets, **kwargs):
    LDA.kwargs = self.kwargs_construct(**kwargs)
    LDA.DataSets = DataSets
    LDA.model = LinearDiscriminantAnalysis()  # asssume spseudolinear is its default setting
    LDA.meanSS = 0  # initialize the self-defined critirion meanSS
    
  def kwargs_construct(self, **kwargs):
    defaultKwargs = {'Learner': 1,                         # LDA_Leaner
                     'LearnerIter': 0, "Replicates": 3     # LDA_Iter                           
                     }
    kwargs = { **defaultKwargs, **kwargs}  # update the args using input_args
    return kwargs   

  def LDA_Leaner(self, z_train,y_train, z_unlabel, ind_unlabel, Y):
    """
      run this function when Learner set to 1.
      embedding via partial label, then learn unknown label via LDA.
    """  
    lda = copy.deepcopy(self)

    clf = self.model
    clf.fit(z_train,y_train)
    train_acc = clf.score(z_train,y_train)
      
    if type(z_unlabel) == np.ndarray:
      # predict_probas include probabilities for all classes for each node
      pred_probas = clf.predict_proba(z_unlabel)
      # assign the classes with the highest probability
      pred_class = clf.predict(z_unlabel)
      # the corresponding probabilities of the predicted classes
      pred_class_prob = pred_probas[range(len(pred_probas)),pred_class] 
    
    Y[ind_unlabel, 0] = pred_class
    lda.Y = Y
    return lda
    

  def LDA_Iter(self, z_train,y_train, z_unlabel, ind_unlabel, Y):
    """
      run this function when Learner set to 1, and LeanerIter is True(>=1)
      ramdonly assign labels to the unknownlabel.
      embedding via partial label, then learn unknown label via LDA.
    """  
    lda = copy.deepcopy(self)

    clf = self.model
    clf.fit(z_train,y_train)
    train_acc = clf.score(z_train,y_train)
      
    if type(z_unlabel) == np.ndarray:
      # predict_probas include probabilities for all classes for each node
      pred_probas = clf.predict_proba(z_unlabel)
      # assign the classes with the highest probability
      pred_class = clf.predict(z_unlabel)
      # the corresponding probabilities of the predicted classes
      pred_class_prob = pred_probas[range(len(pred_probas)),pred_class] 
    
    Y[ind_unlabel, 0] = pred_class
    lda.Y = Y
    return lda  


############-----------------LDA_end-----------------------------###############

############------------Clustering_start-------------------------###############
class Clustering:
  """
    The input DataSets.X is the s*3 edg list
    The innput DataSets.Y can be:
    1. A given cluster size, e.g. [3], meaning in total 3 clusters
    2. A range of cluster sizes. e.g. [3-5], meaning there are possibly 3 to 5 clusters 

  """
  def __init__(self, DataSets, **kwargs):
    self.kwargs = self.kwargs_construct(**kwargs)
    self.DataSets = DataSets
    self.cluster_size_range = self.cluster_size_check() 
    self.K = DataSets.Y[0]
  

  def kwargs_construct(self, **kwargs):
    defaultKwargs = {'Correlation': True,'MaxIter': 50, 'MaxIterK': 5,'Replicates': 3}
    kwargs = { **defaultKwargs, **kwargs}
    return kwargs

  def cluster_size_check(self):
    DataSets = self.DataSets
    Y = DataSets.Y

    cluster_size_range = None # in case that Y is an empty array.

    if len(Y) == 1:
      cluster_size_range = False  # meaning the cluster size is known. e.g. [3]
    if len(Y) > 1:
      cluster_size_range = True   # meaning only know the rane of cluster size. e.g. [2, 3, 4, 5] 
    
    return cluster_size_range
    
  def graph_encoder_cluster(self, K):
    """
      clustering function
    """
    DataSets = self.DataSets
    X = DataSets.X
    n = DataSets.n
    kwargs = self.kwargs


    minSS=-1
    Z = None
    W = None

    for i in range(kwargs['Replicates']):
      Y_temp = np.random.randint(K,size=(n,1))
      for r in range(kwargs['MaxIter']):
        [Zt,Wt] = multi_graph_encoder_embed(DataSets, Y_temp)  
        
        if DataSets.attributes:
          # add U to Z side by side
          Zt = np.concatenate((Zt, DataSets.U), axis=1)
        kmeans = KMeans(n_clusters=K, max_iter = kwargs['MaxIter']).fit(Zt)
        labels = kmeans.labels_ # shape(n,)
        # sum_in_cluster = kmeans.inertia_ # sum of distance within cluster (k,1)
        dis_to_centors = kmeans.transform(Zt)
        # adjusted_rand_score() needs the shape (n,)
        if adjusted_rand_score(Y_temp.reshape(-1,), labels) == 1:
          break
        else:
          # we need labels to be the same shape as for Y(n,1) when assign
          Y_temp = labels.reshape(-1,1) 
      
      # calculate score and compare with meanSS
      tmp = self.temp_score(dis_to_centors, K, labels, n)
      if (minSS == -1) or tmp < minSS:
        Z = Zt
        W = Wt
        minSS = tmp
        Y = labels
    return  Z, Y, W, minSS


  def temp_score(self, dis_to_centors, K, labels, n):
    """
      calculate:
      1. sum_in_cluster(1*k): the sum of the distance from the nodes to the centroid 
      of its belonged cluster
      2. sum_in_cluster_norm(1*k): normalize the sum_in_cluster by the 
      corresponding label count (how many nodes in each cluster)
      3. sum_not_in_cluster(1*k): the sum of the distance of the cluster 
      centroid to the nodes that do not belong to the cluster
      4. sum_not_in_cluster_norm(1*k): normalize the sum_other_centroids by the 
      counts of the nodes that do not belong to the cluster.
      5. temp score(1*k): 
      (normalized sum in cluster / normalized sum not in cluster ) *
      (label count in cluster / total node number)
      6. get mean + 2 standard deviation of temp score, then return 
    """
    label_count = np.bincount(labels)
    sum_in_cluster_squre = np.zeros((K,))

    dis_to_centors_squre = dis_to_centors**2

    for i in range(n):
      label = labels[i]
      sum_in_cluster_squre[label] += dis_to_centors_squre[i][label]
    
    # how to find out if the distance is squared, the current method doesn't do square root.
    sum_not_in_cluster = (np.sum(dis_to_centors_squre, axis=0) - sum_in_cluster_squre)**0.5

    sum_not_in_cluster_norm = sum_not_in_cluster/(n - label_count)
    sum_in_cluster_norm = sum_in_cluster_squre**0.5/label_count

    tmp = sum_in_cluster_norm / sum_not_in_cluster_norm * label_count / n
    tmp = np.mean(tmp) + 2*np.std(tmp)

    return tmp


  def cluster_main(self):
    K = self.K
    DataSets = self.DataSets    
    X = DataSets.X
    n = DataSets.n

    kmax = np.amax(K)
    if n/kmax < 30:
      print('Too many clusters at maximum. Result may bias towards large K. Please make sure n/Kmax >30.')
    # when the cluster size is specified
    if not self.cluster_size_range:
      [Z,Y,W,meanSS]= self.graph_encoder_cluster(K[0])
    # when the range of cluster size is provided 
    # columns are less than n/2 and kmax is less than max(n/2, 10)
    if self.cluster_size_range:
      k_range = len(K)
      if k_range < n/2 and kmax < max(n/2, 10):
          minSS = -1
          Z = 0
          W = 0
          meanSS = np.zeros((k_range,1))
          for i in range(k_range):
            [Zt,Yt,Wt,tmp]= self.graph_encoder_cluster(K[i])
            meanSS[i,0] = i
            if (minSS == -1) or tmp < minSS:
              minSS = tmp
              Y = Yt
              Z = Zt
              W = Wt
    return Z, Y, W, meanSS

############------------Clustering_end---------------------------###############
############------------Evaluation_start---------------------------#############
class Evaluation:
  def GNN_supervise_test(self, gnn, z_test, y_test):
    """
      test the accuracy for GNN_direct
    """
    y_test_one_hot = to_categorical(y_test) 
    # set verbose to 0 to silent the output
    test_loss, test_acc = gnn.model.evaluate(z_test,  y_test_one_hot, verbose=0) 

    return test_acc
  
  def GNN_semi_supervised_learn_test(self,Y_result, Y_original):
    """
      test accuracy for semi-supervised learning
    """
    test_acc = metrics.accuracy_score(Y_result, Y_original)   

    return test_acc

  def GNN_semi_supervised_not_learn_test(self, gnn, Dataset, case):
    """
      test accuracy for semi-supervised learning
    """

    ind_unlabel = Dataset.ind_unlabel
    z_unlabel =  Dataset.z_unlabel 
    y_unlabel_ori = case.Y_ori[ind_unlabel, 0]
    y_unlabel_ori_one_hot = to_categorical(y_unlabel_ori) 
    test_loss, test_acc = gnn.model.evaluate(z_unlabel, y_unlabel_ori_one_hot, verbose=0)

    return test_acc

  def clustering_test(self, Y_result, Y_original):
    """
      test accuracy for semi-supervised learning
    """
    ari = adjusted_rand_score(Y_result, Y_original.reshape(-1,))

    return ari


#Packages for Drive Files

In [4]:
# import packages
## for mount drive purpose
import os
from google.colab import drive

#Mount Drive

In [6]:
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/Graph_ML/semi_dr.shen')

Mounted at /content/drive/


# import ipynb packages

In [7]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=7443b215b91b9740f0cfb1aaef04a01fac4ed0ea33221742412e6876d30c256f
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [8]:
!ls

 Graph_Embed_Cluster_Semi_supervise.ipynb   run.ipynb
'GraphEncoder(Dr.Shen).ipynb'		    semi_supervised_learning.ipynb
 Results.ipynb				    test_cases.ipynb


In [9]:
import import_ipynb
from test_cases import Model, Case

importing Jupyter notebook from test_cases.ipynb


# Test Cases 

## Graph Encoder test case


A = 

\begin{bmatrix}
0 & 1 & 1 & 1 & 0\\
1 & 0 & 1 & 1 & 1\\
1 & 1 & 0 & 1 & 1\\
1 & 1 & 1 & 0 & 1\\
0 & 1 & 1 & 1 & 0
\end{bmatrix}

Labels = [0,0,0,1,1] 


In [21]:
class Encoder_case:
  def __init__(self, A,Y,n):
    Encoder_case.X = A
    Encoder_case.Y = Y
    Encoder_case.n = n

In [22]:
A = np.ones((5,5))
A[0,4] = 0
A[4,0] = 0
np.fill_diagonal(A, 0)

Y = np.array([[0,0,0,1,1]]).reshape((5,1))

print(A)
print(Y)

Encoder_case = Encoder_case(A,Y,5)

[[0. 1. 1. 1. 0.]
 [1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 1.]
 [0. 1. 1. 1. 0.]]
[[0]
 [0]
 [0]
 [1]
 [1]]


### Laplacian = False, correclation = False, DiagA = False

In [23]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0., 1., 1.],
       [0., 2., 1.],
       [0., 3., 1.],
       [1., 0., 1.],
       [1., 2., 1.],
       [1., 3., 1.],
       [1., 4., 1.],
       [2., 0., 1.],
       [2., 1., 1.],
       [2., 3., 1.],
       [2., 4., 1.],
       [3., 0., 1.],
       [3., 1., 1.],
       [3., 2., 1.],
       [3., 4., 1.],
       [4., 1., 1.],
       [4., 2., 1.],
       [4., 3., 1.]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [24]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[1.33333333 1.        ]
 [1.33333333 2.        ]
 [1.33333333 2.        ]
 [2.         1.        ]
 [1.33333333 1.        ]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


### Laplacian = False, correclation = True, DiagA = False

In [26]:
Dataset = DataPreprocess(Encoder_case, Laplacian = False, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0., 1., 1.],
       [0., 2., 1.],
       [0., 3., 1.],
       [1., 0., 1.],
       [1., 2., 1.],
       [1., 3., 1.],
       [1., 4., 1.],
       [2., 0., 1.],
       [2., 1., 1.],
       [2., 3., 1.],
       [2., 4., 1.],
       [3., 0., 1.],
       [3., 1., 1.],
       [3., 2., 1.],
       [3., 4., 1.],
       [4., 1., 1.],
       [4., 2., 1.],
       [4., 3., 1.]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [27]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = True)
print(Z)
print(W)

[[0.8        0.6       ]
 [0.5547002  0.83205029]
 [0.5547002  0.83205029]
 [0.89442719 0.4472136 ]
 [0.8        0.6       ]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


### Laplacian = True, correclation = False, DiagA = False

In [28]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0.        , 1.        , 0.14433757],
       [0.        , 2.        , 0.14433757],
       [0.        , 3.        , 0.14433757],
       [1.        , 0.        , 0.14433757],
       [1.        , 2.        , 0.125     ],
       [1.        , 3.        , 0.125     ],
       [1.        , 4.        , 0.14433757],
       [2.        , 0.        , 0.14433757],
       [2.        , 1.        , 0.125     ],
       [2.        , 3.        , 0.125     ],
       [2.        , 4.        , 0.14433757],
       [3.        , 0.        , 0.14433757],
       [3.        , 1.        , 0.125     ],
       [3.        , 2.        , 0.125     ],
       [3.        , 4.        , 0.14433757],
       [4.        , 1.        , 0.14433757],
       [4.        , 2.        , 0.14433757],
       [4.        , 3.        , 0.14433757]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [29]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[0.19245009 0.14433757]
 [0.17955838 0.26933757]
 [0.17955838 0.26933757]
 [0.26289171 0.14433757]
 [0.19245009 0.14433757]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


### Laplacian = True, correclation = True, DiagA = False

In [30]:
Dataset = DataPreprocess(Encoder_case, Laplacian = True, DiagA = False)
print(Dataset.X)
print(Dataset.Y)
print(Dataset.n)

[array([[0.        , 1.        , 0.14433757],
       [0.        , 2.        , 0.14433757],
       [0.        , 3.        , 0.14433757],
       [1.        , 0.        , 0.14433757],
       [1.        , 2.        , 0.125     ],
       [1.        , 3.        , 0.125     ],
       [1.        , 4.        , 0.14433757],
       [2.        , 0.        , 0.14433757],
       [2.        , 1.        , 0.125     ],
       [2.        , 3.        , 0.125     ],
       [2.        , 4.        , 0.14433757],
       [3.        , 0.        , 0.14433757],
       [3.        , 1.        , 0.125     ],
       [3.        , 2.        , 0.125     ],
       [3.        , 4.        , 0.14433757],
       [4.        , 1.        , 0.14433757],
       [4.        , 2.        , 0.14433757],
       [4.        , 3.        , 0.14433757]])]
[[0]
 [0]
 [0]
 [1]
 [1]]
5


In [31]:
Z, W = graph_encoder_embed(Dataset.X[0], Dataset.Y, Dataset.n, Correlation = False)
print(Z)
print(W)

[[0.19245009 0.14433757]
 [0.17955838 0.26933757]
 [0.17955838 0.26933757]
 [0.26289171 0.14433757]
 [0.19245009 0.14433757]]
[[0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.33333333 0.        ]
 [0.         0.5       ]
 [0.         0.5       ]]


## Set Case 

In [10]:
n = 3000
case = Case(n)

##Clustering

### Case 10

In [ ]:
case_10_cluster = case.case_10_cluster()
case_10_cluster.summary()

name:

    SBM with 3 classes for clustering
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(1, 1)
[[3]]


In [ ]:
print(case_10_cluster.bd)

0.13


In [ ]:
Run(case_10_cluster, "c")

ARI:  0.8036759717543803


### Case 11

In [ ]:
case_11_cluster = case.case_11_cluster()
case_11_cluster.summary()

name:

    SBM with 5 classes for clustering
    
n:
<class 'int'>
3000
d:
<class 'int'>
5
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(1, 1)
[[5]]


In [ ]:
print(case_11_cluster.bd)

0.2


In [ ]:
Run(case_11_cluster, "c")

ARI:  1.0


### case 20

In [ ]:
case_20_cluster = case.case_20_cluster()
case_20_cluster.summary()

name:

    DC-SBM with 3 classes for clustering
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(1, 1)
[[3]]


In [ ]:
print(case_20_cluster.bd)

[0.9, 0.5, 0.2]


In [ ]:
Run(case_20_cluster, "c")

ARI:  0.6710219123774864


### Case 21

In [ ]:
case_21_cluster = case.case_21_cluster()
case_21_cluster.summary()

name:

    DC-SBM with 10 classes for clustering.
    Edge list version. 
    
n:
<class 'int'>
3000
d:
<class 'int'>
10
X:
(30487, 3)
[[   0    3    1]
 [   0  168    1]
 [   0  551    1]
 ...
 [2952 2993    1]
 [2975 2980    1]
 [2983 2987    1]]
Y:
(1, 1)
[[10]]


In [ ]:
print(case_21_cluster.bd)

0.9


In [ ]:
Run(case_21_cluster, "c")

ARI:  0.43355806469613173


## Semi-GNN-learner 2

### case 10

In [18]:
case_10 = case.case_10()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with 95% unknown labels.
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [20]:
print(case_10.bd)

0.13


In [23]:
Run(case_10, "se", Learner = 2, LearnerIter = 10)

acc:  0.5123333333333333


### case 11

In [ ]:
case_11 = case.case_11()
case_11.summary()

name:

    SBM with 5 classes and defined probabilities with 95% unknown labels.  
    
n:
<class 'int'>
3000
d:
<class 'int'>
5
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [ ]:
print(case_11.bd)

0.2


In [ ]:
Run(case_11, "se", Learner = 2, LearnerIter = 10)

acc:  0.6726666666666666


### Case 20

In [ ]:
case_20 = case.case_20()
case_20.summary()

name:

    DC-SBM with 3 classes and defined probabilities with 95% unknown labels.
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [ ]:
print(case_20.bd)

[0.9, 0.5, 0.2]


In [ ]:
Run(case_20, "se", Learner = 2, LearnerIter = 10)

acc:  0.907


### Case 21

In [ ]:
case_21 = case.case_21()
case_21.summary()

name:

    DC-SBM with 10 classes and defined probabilities with 95% unknown labels.
    Edge list version.     
    
n:
<class 'int'>
3000
d:
<class 'int'>
10
X:
(30487, 3)
[[   0    3    1]
 [   0  168    1]
 [   0  551    1]
 ...
 [2952 2993    1]
 [2975 2980    1]
 [2983 2987    1]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [ 5]]


In [ ]:
print(case_21.bd)

0.9


In [ ]:
Run(case_21, "se", Learner = 2, LearnerIter = 10)

acc:  0.852


## Semi-GNN-learner 0

### case 10

In [23]:
case_10 = case.case_10()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with 95% unknown labels.
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [24]:
print(case_10.bd)

0.13


In [25]:
Run(case_10, "se", Learner = 0, LearnerIter = 0)

acc:  0.5261495113372803


### case 11

In [26]:
case_11 = case.case_11()
case_11.summary()

name:

    SBM with 5 classes and defined probabilities with 95% unknown labels.  
    
n:
<class 'int'>
3000
d:
<class 'int'>
5
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [29]:
print(case_11.bd)

0.2


In [28]:
Run(case_11, "se", Learner = 0, LearnerIter = 0)

acc:  0.6410256624221802


### case 20

In [30]:
case_20 = case.case_20()
case_20.summary()

name:

    DC-SBM with 3 classes and defined probabilities with 95% unknown labels.
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [31]:
print(case_20.bd)

[0.9, 0.5, 0.2]


In [32]:
Run(case_20, "se", Learner = 0, LearnerIter = 0)

acc:  0.6356616616249084


### case 21

In [12]:
case_21 = case.case_21()
case_21.summary()

name:

    DC-SBM with 10 classes and defined probabilities with 95% unknown labels.
    Edge list version.     
    
n:
<class 'int'>
3000
d:
<class 'int'>
10
X:
(30487, 3)
[[   0    3    1]
 [   0  168    1]
 [   0  551    1]
 ...
 [2952 2993    1]
 [2975 2980    1]
 [2983 2987    1]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [ 5]]


In [34]:
print(case_21.bd)

0.9


In [35]:
Run(case_21, "se", Learner = 0, LearnerIter = 0)

acc:  0.33778560161590576


## Supervised

### case 10

In [26]:
case_10 = case.case_10_fully_known()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
2400
d:
<class 'int'>
3
X:
(2400, 2400)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(2400, 1)
[[2]
 [2]
 [0]
 ...
 [0]
 [2]
 [2]]


In [14]:
print(case_10.bd)

0.13


In [27]:
Run(case_10, "su")

acc:  0.8183333277702332


### case 11

In [16]:
case_11 = case.case_11_fully_known()
case_11.summary()

name:

    SBM with 5 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
2400
d:
<class 'int'>
5
X:
(2400, 2400)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(2400, 1)
[[3]
 [4]
 [0]
 ...
 [0]
 [2]
 [3]]


In [17]:
print(case_11.bd)

0.2


In [18]:
Run(case_11, "su")

acc:  0.996666669845581


### case 20

In [19]:
case_20 = case.case_20_fully_known()
case_20.summary()

name:

    DC-SBM with 3 classes and defined probabilities with fully known labels
    80% for training and 20% for testing
    
n:
<class 'int'>
2400
d:
<class 'int'>
3
X:
(2400, 2400)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(2400, 1)
[[2]
 [2]
 [0]
 ...
 [0]
 [2]
 [2]]


In [20]:
print(case_20.bd)

[0.9, 0.5, 0.2]


In [21]:
Run(case_20, "su")

acc:  0.5066666603088379


### case 21

In [22]:
case_21 = case.case_21_fully_known()
case_21.summary()

name:

    DC-SBM with 10 classes and defined probabilities with fully known labels.
    Edge list version. 
    
n:
<class 'int'>
2400
d:
<class 'int'>
10
X:
(39410, 3)
[[   0   98    1]
 [   0  318    1]
 [   0  333    1]
 ...
 [2397 2193    1]
 [2397 2249    1]
 [2397 2297    1]]
Y:
(2400, 1)
[[6]
 [9]
 [1]
 ...
 [1]
 [5]
 [6]]


In [23]:
print(case_21.bd)

0.9


In [24]:
print(case_21.X_test)

[[   0   87    1]
 [   0   93    1]
 [   0  538    1]
 ...
 [ 599 1817    1]
 [ 599 2149    1]
 [ 599 2287    1]]


In [25]:
Run(case_21, "su")

acc:  0.15000000596046448


#### Semi

In [ ]:
print(case_11.bd)

0.2


In [13]:
case_10 = case.case_10()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with 95% unknown labels.
    
n:
<class 'int'>
100
d:
<class 'int'>
3
X:
(100, 100)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
Y:
(100, 1)
[[-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 2]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [ 1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 2]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 2]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]]


In [14]:
Dataset = DataPreprocess(case_10)
Dataset = Dataset.semi_supervise_preprocess()
print(Dataset.z_unlabel)
# kwargs_for_gnn =  {'Learner': 0, 'LearnerIter': 10}
# gnn = GNN(Dataset, **kwargs_for_gnn)
# Results = gnn.GNN_complete()

# results = metrics.accuracy_score(case_11.Y_ori, Results.Y)
# print(results)

[[0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         0.         0.        ]
 [0.         0.83205029 0.5547002 ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]
 [0.         1.         0.        ]
 [0.         0.         1.        ]
 [0.         0.         1.        ]
 [0.         0.83205029 0.5547002 ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         1.         0.  

## semi-GNN-Learner 2 for 5% unknown

### case 10

In [11]:
case_10 = case.case_10_1()
case_10.summary()

name:

    SBM with 3 classes and defined probabilities with 5% unknown labels.
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
Y:
(3000, 1)
[[1]
 [0]
 [2]
 ...
 [2]
 [2]
 [2]]


In [12]:
print(case_10.bd)

0.13


In [13]:
Run(case_10, "se", Learner = 2, LearnerIter = 10)

acc:  0.998


# Others

### Semi

**acc 0.912**

In [ ]:
case_20 = case.case_20()
case_20.summary()

name:

    DC-SBM with 3 classes and defined probabilities with 95% unknown labels
    The unkown labels maybe less than 95% in order to make sure that 
    at least there is one node belongs to one label     
    
n:
<class 'int'>
3000
d:
<class 'int'>
3
X:
(3000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


In [ ]:
print(case_20.bd)

[0.9, 0.5, 0.2]


In [ ]:
Dataset = DataPreprocess(case_20.X, case_20.Y)
Dataset = Dataset.semi_supervise_preprocess()
kwargs_for_gnn =  {'Learner': 2, 'LearnerIter': 10}
gnn = GNN(Dataset, **kwargs_for_gnn)
Results = gnn.GNN_complete()

results = metrics.accuracy_score(case_20.Y_ori, Results.Y)
print(results)

0.912


### case 21
DC-SBM, 10 classes

#### Semi

## Semi-LDA-Learner 1

In [ ]:
case_21 = case.case_21()
case_21.summary()

name:

    DC-SBM with 10 classes and defined probabilities with 95% unknown labels
    The unkown labels maybe less than 95% in order to make sure that 
    at least there is one node belongs to one label       
    
n:
<class 'int'>
3000
d:
<class 'int'>
10
X:
(30487, 3)
[[   0    3    1]
 [   0  168    1]
 [   0  551    1]
 ...
 [2952 2993    1]
 [2975 2980    1]
 [2983 2987    1]]
Y:
(3000, 1)
[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]
